---

Created for [Pricing and Hedging Derivative Securities: Theory and Methods](https://book.derivative-securities.org/)

Authored by
- Kerry Back, Rice University
- Hong Liu, Washington University in St. Louis
- Mark Loewenstein, University of Maryland
 
---

<a target="_blank" href="https://colab.research.google.com/github/math-finance-book/book-code/blob/main/12_MonteCarlo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:

import plotly
from IPython.display import display, HTML

plotly.offline.init_notebook_mode(connected=True)
display(
    HTML(
        '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
    )
)

In [ ]:
# Simulate Geometric Brownian Motion
import numpy as np
import matplotlib.pyplot as plt
# number of paths
n = 10000
#number of divisions
m = 1
# Interest rate (We set the drift equal to the interest rate for the risk-neutral probability)
r = 0.1
# Volatility
sig = 0.2
# Initial Stock Price
S0 = 42
# Maturity
T = 0.5
#Strike Price
K=40
# Dividend Yield
q=0.0
# Delta t
dt = T/m
# Drift
drift = (r-q-0.5*sig**2)
# Volatility
vol = sig * np.sqrt(dt)

t = np.array(range(0,m + 1,1)) * dt

# seed for random generator
seed= 2020
# define a random generator
np.random.seed(seed)
inc = np.zeros(shape = (m + 1, n))
inc[1:] = np.transpose(np.random.normal(loc = 0, scale = vol,size = (n,m)))
St = np.zeros(shape = (m + 1, n))
St = S0 * np.exp(np.cumsum(inc,axis=0) + (drift * t[0:m + 1])[:,None])
St1 = S0 * np.exp(-np.cumsum(inc,axis=0) + (drift * t[0:m + 1])[:,None])

In [ ]:
cc=np.maximum(St[m,:]-K,0)
cp = np.mean(cc) * np.exp(-r * T)
cc1=np.maximum(St1[m,:]-K,0)*np.exp(-r * T)
cp1= np.mean(np.maximum(St1[m,:]-K,0)) * np.exp(-r * T)

print('The first sample gives an estimated call price=',cp)
print('The second sample gives an estimated call price=',cp1)
bsc = (cp+cp1)/2
print('The average of the two estimates=',bsc)

In [ ]:
from scipy.stats import norm
import numpy as np
from scipy.optimize import minimize, minimize_scalar

def blackscholes(S0, K, r, q, sig, T, call = True):
    '''Calculate option price using B-S formula.
    
    Args:
        S0 (num): initial price of underlying asset.
        K (num): strick price.
        r (num): risk free rate
        q (num): dividend yield
        sig (num): Black-Scholes volatility.
        T (num): maturity.
        call (bool): True returns call price, False returns put price.
        
    Returns:
        num
    '''
    d1 = (np.log(S0/K) + (r -q + sig**2/2) * T)/(sig*np.sqrt(T))
    d2 = d1 - sig*np.sqrt(T)
    if call:
        return np.exp(- q *T) * S0 * norm.cdf(d1,0,1) - K * np.exp(-r * T) * norm.cdf(d2,0, 1) 
    else:
        return -np.exp(-q * T) * S0 * norm.cdf(-d1,0,1) + K * np.exp(-r * T) * norm.cdf(-d2,0, 1)

truebsc=blackscholes(S0, K, r, q, sig, T, call = True)
print('The black scholes fromula=',truebsc)

In [ ]:
SS=np.mean(St[m,:])*np.exp(-r*T)
print('The Estimated Stock Price for the first sample is =', SS)
print('The actual stock price should be=', S0)
print('The error is =', S0-SS)

In [ ]:
hatbeta= np.cov(St[m,:],cc)[0,1]/np.cov(St[m,],cc)[1,1]
hatbeta1=np.cov(St1[m,:],cc1)[0,1]/np.cov(St1[m,],cc1)[1,1]
correction =hatbeta*(S0-SS)
update=cp + correction
print('hatbeta=',hatbeta)
print('The original estimate for the call price from the first sample=',cp)
print('The original estimate for the call price from the second sample=',cp1)
print('The updated estimate from the first sample is=',update)
SS1=np.mean(St1[m,:])*np.exp(-r*T)
update1=cp1+hatbeta1*(S0-SS1)
print('The updated estimate from the second sample is=',update1)
print('The average of the updated estimates =',(update+update1)/2)

In [ ]:
print('The exact Black Scholes Price is=', truebsc)

In [ ]:
# Simulate Geometric Brownian Motion
import numpy as np
import matplotlib.pyplot as plt
# number of paths
n = 1000
#number of divisions
m = 1000
# Interest rate (We set the drift equal to the interest rate for the risk-neutral probability)
r = 0.1
# Dividend yield
q=0.0
# Volatility
sig = 0.2
# Initial Stock Price
S0 = 42
# Maturity
T = 0.5
#Strike Price
K=40
# Delta t
dt = T/m
# Drift
drift = (r-q-0.5*sig**2)
# Volatility
vol = sig * np.sqrt(dt)

t = np.array(range(0,m + 1,1)) * dt

# seed for random generator
seed= 2024
# define a random generator
np.random.seed(seed)
inc = np.zeros(shape = (m + 1, n))
inc[1:] = np.transpose(np.random.normal(loc = 0, scale = vol,size = (n,m)))
St = np.zeros(shape = (m + 1, n))
St = S0 * np.exp(np.cumsum(inc,axis=0) + (drift * t[0:m + 1])[:,None])
St1 = S0 * np.exp(-np.cumsum(inc,axis=0) + (drift * t[0:m + 1])[:,None])

In [ ]:
def floating_strike_call(S, r, sigma, q, T, SMin):
    d1 = (np.log(S / SMin) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    d2prime = (np.log(SMin / S) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    N1 = norm.cdf(d1)
    N2 = norm.cdf(d2)
    N2prime = norm.cdf(d2prime)
    x = 2 * (r - q) / (sigma ** 2)
    return np.exp(-q * T) * S * N1 - np.exp(-r * T) * SMin * N2 + (1 / x) * (SMin / S) ** x * np.exp(-r * T) * SMin * N2prime - (1 / x) * np.exp(-q * T) * S * (1 - N1)

S = 100
r = 0.05
sigma = 0.2
q = 0.02
T=1

Stmin=St[m:]-np.minimum(np.min(St,axis=0),S0)
St1min=St1[m:]-np.minimum(np.min(St1,axis=0),S0)
floatlkbk=np.exp(-r*T)*np.mean(Stmin)
floatlkbk1=np.exp(-r*T)*np.mean(St1min)

print('The first estimate is=',floatlkbk)
print('The second estimate is=',floatlkbk1)
print('The average estimate is=',(floatlkbk+floatlkbk1)/2)
print('The exact formula is=',floating_strike_call(S0, r, sigma, 0, T, S0))

In [ ]:
Stmax=np.maximum(np.max(St,axis=0)-K,0)
St1max=np.maximum(np.max(St1,axis=0)-K,0)
lookbck = np.exp(-r*T) *np.mean(Stmax)
lookbck1=np.exp(-r * T)*np.mean(St1max)
print('The first estimate is=',lookbck)
print('The second estimate is=',lookbck1)
print('The average estimate is=', (lookbck + lookbck1)/2)

In [ ]:
average = np.mean(St,axis=0)
average1 = np.mean(St1,axis=0)
dpayoff=np.exp(-r*T)*np.mean(np.maximum(average-K,0))
dpayoff1=np.exp(-r*T)*np.mean(np.maximum(average1-K,0))
print('The first estimate is=',dpayoff)
print('The second estimate is=',dpayoff1)
print('The average of the estimates=',(dpayoff+dpayoff1)/2)

In [ ]:
def black_scholes_call(S, K, r, sigma, q, T):
    """
    Inputs:
    S = initial stock price
    K = strike price
    r = risk-free rate
    sigma = volatility
    q = dividend yield
    T = time to maturity
    """
    if sigma == 0:
        return max(0, np.exp(-q * T) * S - np.exp(-r * T) * K)
    else:
        d1 = (np.log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        N1 = norm.cdf(d1)
        N2 = norm.cdf(d2)
        return np.exp(-q * T) * S * N1 - np.exp(-r * T) * K * N2

def discrete_geom_average_price_call(S, K, r, sigma, q, T, N):
    dt = T / N
    nu = r - q - 0.5 * sigma ** 2
    a = N * (N + 1) * (2 * N + 1) / 6
    V = np.exp(-r * T) * S * np.exp(((N + 1) * nu / 2 + sigma ** 2 * a / (2 * N ** 2)) * dt)
    sigavg = sigma * np.sqrt(a) / (N ** 1.5)
    return black_scholes_call(V, K, r, sigavg, q, T)

geom=np.exp((np.mean(np.log(St),axis=0)))
geom1=np.exp((np.mean(np.log(St1),axis=0)))
geomavgpo=np.maximum(geom-K,0)
geomavg1po=np.maximum(geom1-K,0)
value=np.mean(geomavgpo)*np.exp(-r*T)
value1=np.mean(geomavg1po)*np.exp(-r*T)
tga=discrete_geom_average_price_call(S0, K, r, sigma, q, T, m)
error =tga-value
error1=tga-value1
print('The estimate from the first sample=',value)
print('The estimate from the second sample=',value1)
print('The average of the two estimates is=',(value+value1)/2)
print('The value from the exact formula=',tga)
print('The error in the first estimate=',error)
print('The error in the second estimate=',error1)

In [ ]:
incpre = np.zeros(shape = (m + 1, n))
incpre[1:] = np.transpose(np.random.normal(loc = 0, scale = vol,size = (n,m)))
Stpre = np.zeros(shape = (m + 1, n))
St1pre=np.zeros(shape = (m + 1, n))
Stpre = S0 * np.exp(np.cumsum(inc,axis=0) + (drift * t[0:m + 1])[:,None])
St1pre = S0 * np.exp(-np.cumsum(inc,axis=0) + (drift * t[0:m + 1])[:,None])

amean=np.mean(Stpre,axis=0)
amean1=np.mean(St1pre,axis=0)
apo = np.maximum(amean-K,0)
a1po=np.maximum(amean1-K,0)
gmean=np.exp(np.mean(np.log(St),axis=0))
g1mean=np.exp(np.mean(np.log(St1),axis=0))
gpo=np.maximum(gmean-K,0)
g1po=np.maximum(g1mean-K,0)
beta=np.cov(gpo,apo)[0,1]/np.cov(gpo,apo)[1,1]
beta1=np.cov(g1po,a1po)[0,1]/np.cov(g1po,a1po)[1,1]
update=dpayoff +beta*error
update1=dpayoff1+beta1*error1

print('The updated estimate for the first sample=',update)
print('The updated value for the second sample=',update1)
print('The average of the updated values is=',(update +update1)/2)

In [ ]:
# Simulate 2 Geometric Brownian Motions
import numpy as np
import matplotlib.pyplot as plt
# number of paths
n = 1000
#number of divisions
m = 1000
# Interest rate (We set the drift equal to the interest rate for the risk-neutral probability)
r = 0.1
# Dividend yield
q1=0.0
q2=0
# Volatility
sig1 = 0.2
sig2=.3
# correlation
rho=0.5
# Initial Stock Price
S0 = 42
V0 = 50
# Maturity
T = 0.5

# Delta t
dt = T/m
# Drift
drift1 = (r-q1-0.5*sig1**2)
drift2 = (r-q2-0.5*sig2**2)
# Volatility
vol = np.sqrt(dt)

t = np.array(range(0,m + 1,1)) * dt

# seed for random generator
seed= 2024
# define a random generator
np.random.seed(seed)
inc = np.zeros(shape = (m + 1, n))
inc[1:] = np.transpose(np.random.normal(loc = 0, scale = vol,size = (n,m)))
inc1 = np.zeros(shape = (m + 1, n))
inc1[1:] = np.transpose(np.random.normal(loc = 0, scale = vol,size = (n,m)))
incr = np.zeros(shape = (m + 1, n))
incr = rho*inc + np.sqrt(1-rho**2)*inc1

In [ ]:
St1 = np.zeros(shape = (m + 1, n))
St2 = np.zeros(shape = (m + 1, n))
St1 = S0 * np.exp(sig1*np.cumsum(inc,axis=0) + (drift1 * t[0:m + 1])[:,None])
St2 = V0 * np.exp(sig2*np.cumsum(incr,axis=0) + (drift2 * t[0:m + 1])[:,None])

In [ ]:
St1a = np.zeros(shape = (m + 1, n))
St2a = np.zeros(shape = (m + 1, n))
St1a = S0 * np.exp(-sig1*np.cumsum(inc,axis=0) + (drift1 * t[0:m + 1])[:,None])
St2a = V0 * np.exp(-sig2*np.cumsum(incr,axis=0) + (drift2 * t[0:m + 1])[:,None])

In [ ]:
payoff = np.maximum(St1[m,:],St2[m,:])
payoffa = np.maximum(St1a[m,:],St2a[m,:])
value= np.exp(-r*T)*np.mean(payoff)
valuea= np.exp(-r*T)*np.mean(payoffa)

print('The first estmate is =',value)
print('The second estimate is =',valuea)
print('The avergae of the estimates is=',(value+valuea)/2)

In [ ]:
w=0.5
K=45
basketpo=np.maximum(w*St1[m,:]+(1-w)*St2[m,:]-K,0)
basketpoa=np.maximum(w*St1a[m,:]+(1-w)*St2a[m,:]-K,0)
estimate=np.exp(-r*T)*np.mean(basketpo)
estimatea=np.exp(-r*T)*np.mean(basketpoa)
print('The first estimate is =',estimate)
print('The second estimate is =',estimatea)
print('The average of the estimates=',(estimate+estimatea)/2)

In [ ]:
import numpy as np
#risk free rate
r=0.1
# number of assets
k=3

# number of paths
n=100000
# Horizon
T=0.5

# Initial price

S0=[42,50,45]

# Basket Weights
w=[.25,.5,.25]

#Strike Price

K=45

#  put in volatilities
sig1=.2
sig2=.3
sig3=.4

#create diagonal
sig=[sig1,sig2,sig3]

S=np.diag(sig)

# drift of log returns

drift= r*np.ones(k) -0.5*np.dot(S@S,np.ones(k))


# correlation matrix

rho=np.array([[1.0, 0.5, 0.3],
                  [0.5, 1.0, 0.2],
                  [0.3, 0.2, 1.0]])
# covariance matrix

V = S@rho@S

# generate uniform n*k normal uncorrelated random variables

seed=2024
np.random.seed(seed)

inc1=np.transpose(np.random.normal(loc = 0, scale = np.sqrt(T),size = (n,k)))
 


# create correlated random variables
Z=np.linalg.cholesky(V)
incr=np.dot(Z,inc1)

print('The sample correlation matrix =',np.corrcoef(incr))
print('The input correlation matrix =',rho)




St = S0 * np.exp(drift *T + np.transpose(incr))
#antithetic sample
St1 = S0 * np.exp( drift * T - np.transpose(incr))

estimate=np.mean(St,axis=0)*np.exp(-r*T)
estimate1=np.mean(St1,axis=0)*np.exp(-r*T)
print('The average discounted stock price averaged over both samples=',(estimate+estimate1)/2)
print('The initial Srock Price input =',S0)

basketpo=np.maximum(w@np.transpose(St)-K,0)
basketpo1=np.maximum(w@np.transpose(St1)-K,0)
value=np.mean(basketpo)*np.exp(-r*T)
value1=np.mean(basketpo1)*np.exp(-r*T)
print('The first sample estimate of the basket option value=',value)
print('The second sample estimate of the basket option value=',value1)
print('The average estimate of the basket option value=',(value+value1)/2)

In [ ]:
import numpy as np
#number of time steps
m=2
# number of assets
k=3
# number of sample paths
n=100000

#risk free rate
r=0.1



# Horizon
T=0.5
# delta t
dt=T/m
# Initial price

S0=[42,50,45]

#Strike Price

K=45

#  put in volatilities
sig1=.2
sig2=.3
sig3=.4

#create diagonal
sig=[sig1,sig2,sig3]

S=np.diag(sig)


# correlation matrix

rho=np.array([[1.0, 0.5, 0.3],
                  [0.5, 1.0, 0.2],
                  [0.3, 0.2, 1.0]])

# covariance matrix

V = S@rho@S

# drift of log returns
drift= np.array(r*np.ones(k) -0.5*np.dot(S@S,np.ones(k)))*dt

# times vector
t=np.array(range(1,m + 1,1))


driftv = np.transpose(np.kron(drift,t).reshape(3,m))

# generate uniform n(paths)*k(assets)*m(time steps) normal uncorrelated random variables

seed=2024
np.random.seed(seed)

inc=np.random.normal(loc = 0, scale = np.sqrt(dt),size = (n,k,m))

#create correlated random increments
Z=np.linalg.cholesky(V)
# numpy matmul assumes last two define matrix multiplication
incr=np.matmul(Z,inc)



SSt=S0*np.exp(driftv)

# generate returns along path and antithetic path
# first e^{cumsum(increments)} gives e^sigma B_t for different t

Stb = np.exp(np.cumsum(incr[:,:,],axis=2))
Stb1 = np.exp(-np.cumsum(incr[:,:,],axis=2))


#Multiply by S0 e^drift for each t
St=np.multiply(Stb,np.transpose(SSt))
St1=np.multiply(Stb1,np.transpose(SSt))

#Last date returns
Stm=St[:,:,m-1]
Stm1=St1[:,:,m-1]

#define payoff
# Basket Weights
w=[.25,.5,.25]


payoff= np.maximum(np.matmul(Stm,np.transpose(w))-K,0)
payoff1= np.maximum(np.matmul(Stm1,np.transpose(w))-K,0)


value= np.exp(-r*T)*np.mean(payoff)
value1= np.exp(-r*T)*np.mean(payoff1)
print('The estimate for the first sample value=',value)
print('The estimate for the second sample value=',value1)
print('The average estimate for the value=',(value+value1)/2)